# 1. Load & Prepare

In [ ]:
!pip install transformers
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install pororo

In [ ]:
!pip install tensorflow

In [11]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel, AutoModel, AutoTokenizer

import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

# import pororo
# from pororo import Pororo

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [12]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)
print(transformers.__version__)
print(keras.__version__)
print(sklearn.__version__)
# print(pororo.__version__)
print(selenium.__version__)

2.11.0
1.21.6
1.3.5
4.30.2
2.11.0
1.0.2
4.11.2


In [13]:
tf.test.gpu_device_name()

2023-08-03 13:34:27.946551: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-03 13:34:27.946683: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-03 13:34:27.946757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-03 13:34:27.946814: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to down

''

In [19]:
# Setting results path
data_path='/home/aiteam/hj/data'
model_path='/home/aiteam/hj/model'
sub_path='/home/aiteam/hj/sub'

In [20]:
# # Load data
train=pd.read_csv(os.path.join(data_path,'train_data.csv'))
test=pd.read_csv(os.path.join(data_path,'test_data.csv'))
sample_submission=pd.read_csv(os.path.join(data_path,'sample_submission.csv'))

# train = pd.read_csv('train_data.csv')
# test = pd.read_csv('test_data.csv')
# sample_submission = pd.read_csv('sample_submission.csv')

In [6]:
# Main Tokenizer used in RobertaModel
# tokenizer = BertTokenizer.from_pretrained('klue/roberta-large')
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', force_download = True)

# 2. Making Back Translation Data(KOR->ENG)

+ Back translation에 관한 아이디어를 차용해 데이터를 증강합니다. 해당과정에서 google translator나 kakao에서 발표한 Pororo를 사용하려고 했으나, 일일 용량제한이 있거나 실행시간이 지나치게 오래걸리거나 번역기의 성능이 papago를 따라가지 못한다고 판단해 네이버 Papago를 크롤링해 사용하였습니다. 또한 Papago를 사용한 이유중 하나는 해당 데이터는 '...'과 같은 특수 문자들이 사용되는 일종의 규칙이 존재한것을 발견했는데, 이러한 문장 스타일에 대한 규칙을 보존하면서 번역하는 모습을 볼 수 있었습니다. 

+ 그래서 최종적으로 papago를 사용하였으며, 번역되지 않는 문장들의 index를 추출해 pororo로 2차 번역을 하는 과정을 거쳤습니다. 진행하면서 느낀점은 그래도 시간이 오래걸리는 것과, 번역중 문장과 관계없는 글자나 특수문자들이 어그러져서 나오는 등의 문제들이 존재했다는 점입니다(한자가 포함되어있는 문장에서 자주나타남 / 동시에 크롤링 과정에서의 오류로 짐작) 이런 부분들과 번역과정에서 원본 문장 길이에 비해 지나치게 짧은 번역을 하는 부분을 이상번역치로 간주해 Pororo로 재번역을 실행했습니다. 특히 한자에 관한 부분은 높은 빈도로 나오는 한자들을 추출해 한글로 변환한뒤, 재번역했습니다.

In [15]:
def chrome_setting():
  service = Service(executable_path = '/usr/bin/chromedriver')
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  # driver = webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)
  driver = webdriver.Chrome(service = service, options=chrome_options)
  return driver

In [16]:
driver=chrome_setting()

In [21]:
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')


In [22]:
trans_list=[]
kor_to_trans(train['title'], 'en',0,10000)
np.save(data_path+'kor_to_eng_train_0_10000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',10000,20000)
np.save(data_path+'kor_to_eng_train_10000_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',20000,30000)
np.save(data_path+'kor_to_eng_train_20000_30000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',30000,40000)
np.save(data_path+'kor_to_eng_train_30000_40000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',40000,len(train))
np.save(data_path+'kor_to_eng_train_40000_all.npy',trans_list)

  0%|          | 0/10000 [00:00<?, ?it/s]

99th :  A case that was suspended because the head of the Cheong Creative Center was not subject to inspection
198th :  英 Frozen truck tragedy repatriates all 39 Vietnamese bodies
297th :  For the first time in professional basketball history, SK Champion won three consecutive games after losing two...
396th :  Men's basketball with the effect of Ra Gun-ah at the Asian Games...
495th :  Improving the process of acquiring nationality of Jewish descendants who left Nazi victims
594th :  Turkey's military operation in northern Syria...Pushing Turkey into the swamp of the Middle East
693th :  Ministry of Science and ICT's laboratory start-up business contest...a total of 9.3 billion won
792th :  High-ranking Iranian officials, we should also detain British oil tankers
891th :  Choi Soon-sil's return to Korea as soon as possible to President Lee Jung-hyun, breaking news of the active investigation proposal
990th :  Bulletin board Samjeong KPMG 12th IPO Success Strategy Seminar
1089th :  Man

  0%|          | 0/10000 [00:00<?, ?it/s]

10098th :  SK CC KAIST Online Open Course Co-opted
10197th :  If Ryu Hyun-jin catches the Angels, the third Korean big leaguer in MLB...
10296th :  Are you going to make the National Assembly spin to cover the hiring scandal of the Liberty Korea Party…pressure on the acceptance of state aid
10395th :  Lim Chang-yong is about to win the first away game in 13 years on the day of the 1,000th game between South Korea, the U.S. and Japan
10494th :  How about cherry blossom viewing in Japan...Kagoshima first blooming on the 20th of this month
10593th :  SKT President Park Jung-ho will stabilize 5G network quality within two months
10692th :  The Ministry of Culture, Sports and Tourism signed an MOU with three webtoon platforms in Seoul
10791th :  Obama is busy visiting foreign countries in the new year…Attention to the role of domestic presidential election
10890th :  Women's Basketball Shinhan Bank Kim Dan-bi pitches at Gocheok Dome on the 19th
10989th :  This time, Trump's campaign site is

  0%|          | 0/10000 [00:00<?, ?it/s]

20097th :  Jin Kyung-joon's verification hole...禹 There is no legal basis for verifying borrowed-name property
20196th :  Cultural News National Intangible Heritage Center will perform next month
20295th :  Constitutional amendment completed during President Park's term...the public's ambitious constitutional amendment
20394th :  Samsung AI secretary Bixby Chinese service from the 30th
20493th :  President Park returned home from the nuclear summit and Mexico schedule
20592th :  Among the men's and women's volleyball PS teams, Hyundai won the most at home...
20691th :  14 senior citizens, women, youth, etc. are appointed as honorary mayors of Seoul Metropolitan Government
20790th :  The National Election Commission will hold its first mock test of the April 13 general election tomorrow
20889th :  Haynes, fill in the void, Mace...SK's fast basketball worked
20988th :  Racism, embezzlement, sexism...Self-investigation into WHO's internal revelations
21087th :  G-Star Battleground Mobile 

  0%|          | 0/10000 [00:00<?, ?it/s]

30096th :  Kang Jung-ho finally broke up with MLB Pittsburgh...a discharge waiting measure
30195th :  Housing Sales and Jeonse Stabilization in March...Effects of loan regulations, etc
30294th :  SKT Jump AR Zoo Newly Renovated...Four Halloween costume mini-animals released
30393th :  Former head coach Park Young-jin as coach of KDB Life Insurance in Women's Basketball
30492th :  Development of KT 5G wireless backhaul technology
30591th :  Boat subversion with 71 refugees on a lake in eastern Turkey...Seven dead
30690th :  Korea Women's Soccer Asian Cup Korea-Japan 00 draw...a failure to win one's first victory
30789th :  Inter-Korean summit car parade in downtown Pyongyang
30888th :  President Sohn Hak-kyu's apology is sincere, but he does not respond to the people's demands
30987th :  Poems that you enjoy watching, listening to, and enjoying...a visiting poetry performance
31086th :  President Moon's withdrawal GP from the Demilitarized Zone reported the progress of inter-Korean mutu

In [ ]:
eng_data0=np.load(data_path+'kor_to_eng_train_0_10000.npy')
eng_data1=np.load(data_path+'kor_to_eng_train_10000_20000.npy')
eng_data2=np.load(data_path+'kor_to_eng_train_20000_30000.npy')
eng_data3=np.load(data_path+'kor_to_eng_train_30000_40000.npy')
eng_data4=np.load(data_path+'kor_to_eng_train_40000_all.npy')

eng_data=[*eng_data0,*eng_data1,*eng_data2,*eng_data3,*eng_data4]
eng_data=pd.DataFrame(eng_data,columns=['eng_title'])

In [ ]:
back_train=pd.concat([train,eng_data],axis=1)
back_train

## 2.1 Translate Chinese characters into English again

+ We have often identified cases in which translators do not translate sentences containing Chinese characters properly.

+ Therefore, it attempts to re-translate sentences containing Chinese characters with high frequency of appearance.

In [ ]:
# Check Chinese characters with high frequency of appearance
k=[]
for i in range(0,len(train)):
  a=re.findall('[一-龥]',back_train['title'][i])
  if len(a)!=0:
    k=[*k,*a]

Counter(k).most_common()[:35]

In [ ]:
# Save index containing Chinese characters
hanja_ind=[]
for i in range(0,len(train)):
  a=re.findall('[一-龥]',back_train['title'][i])
  if len(a)!=0:
    hanja_ind.append(i)

# Replace to korean
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

d = { "中": "중국", "美": "미국","北":"북한",'日':"일본",'英':'영국','行':'행','靑':'청와대','朴':'박','銀':'은행','與':'여당',
     '文':'문','野':'야당','獨':'독일','伊':'이탈리아','韓':'한국','佛':'프랑스','前':'전','檢':'검찰','軍':'군','安':'안철수','南':'남한',
     '亞':'아시아','展':'전시회','重':'차','株':'주식','詩':'시'}

In [ ]:
kr_title=[]
for i in back_train.iloc[hanja_ind,1]:
  temp=replace_all(i,d)
  kr_title.append(temp)

# Re-translation target (Chinese -> Korean)
print(len(kr_title))

In [ ]:
# Reset selenium chrome driver
driver=chrome_setting()

In [ ]:
def kor_to_trans_again(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kr_title.npy',trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
trans_list=[]
kor_to_trans_again(kr_title, 'en',0,len(kr_title))
np.save(data_path+'kr_title.npy',trans_list)

In [ ]:
kr_title=np.load(data_path+'kr_title.npy')
back_train.at[hanja_ind,'eng_title']=kr_title

In [ ]:
back_train.iloc[hanja_ind,:]

## 2.2 Retry Untranslated Sentences

In [ ]:
# Translated sentences containing Korean : 3 case
hangul_ind=[]
for i in range(0,len(back_train)):
  temp=re.findall('[가-힣]',back_train['eng_title'][i])
  if len(temp)!=0:
    hangul_ind.append(i)

hangul_ind

In [ ]:
back_train.iloc[hangul_ind,3]

In [ ]:
# Modify manually
back_train.at[2341,'eng_title']='The governments comprehensive financial sanctions on dozens of North Korean organizations and individuals.'
back_train.at[9461,'eng_title']=back_train['eng_title'][9461].replace('궤멸하다','').replace('  ','')
back_train.at[39198,'eng_title']=back_train['eng_title'][39198].replace('기겁하다','').replace('  ','')

### Using pororo because it is not translated using Papago

In [ ]:
back_train[back_train['eng_title']=='']

In [ ]:
# Convert Chinese characters into Korean for sentences that have not been translated.
not_trans_index=back_train[back_train['eng_title']=='']['title'].index

not_transed=[]
for i in back_train.iloc[not_trans_index,1]:
  temp=replace_all(i,d)
  not_transed.append(temp)

In [ ]:
# Retranslating using pororo
mt = Pororo(task="translation", lang="multi")

not_trans=[]

for i in tqdm(range(0,len(not_transed))):
  not_trans.append(mt(not_transed[i], src="ko", tgt="en"))
  if i%100==0:
    np.save(data_path+'not_trans.npy',not_trans)
    print('{}th : '.format(i),not_trans[i])

np.save(data_path+'not_trans.npy',not_trans)

In [ ]:
not_trans=np.load(data_path+'not_trans.npy')
back_train.at[not_trans_index,'eng_title']=not_trans

In [ ]:
back_train.iloc[not_trans_index,:]

In [ ]:
back_train.to_csv(data_path+'back_train.csv',index=False)

# 3. Making Back Translation Data(ENG->KOR)

In [ ]:
# Reset selenium chrome driver
driver=chrome_setting()

In [ ]:
def trans_to_kor(transed_list, transed_lang,start_index,final_index): 
  
  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(1.5)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      driver.get('https://papago.naver.com/?sk=en&tk='+transed_lang+'&st='+transed_list[i])
      time.sleep(2)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.2)
      backtrans = element.text 

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except:
      trans_list.append('')

In [ ]:
trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',0,10000)
np.save(data_path+'eng_to_kor_train_0_10000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',10000,20000)
np.save(data_path+'eng_to_kor_train_10000_20000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',20000,30000)
np.save(data_path+'eng_to_kor_train_20000_30000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',30000,40000)
np.save(data_path+'eng_to_kor_train_30000_40000.npy',trans_list)

trans_list=[]
trans_to_kor(back_train['eng_title'], 'ko',40000,len(back_train))
np.save(data_path+'eng_to_kor_train_40000_all.npy',trans_list)

## 3.1 Retry Untranslated Sentences

In [ ]:
back0=np.load(data_path+'eng_to_kor_train_0_10000.npy')
back1=np.load(data_path+'eng_to_kor_train_10000_20000.npy')
back2=np.load(data_path+'eng_to_kor_train_20000_30000.npy')
back3=np.load(data_path+'eng_to_kor_train_30000_40000.npy')
back4=np.load(data_path+'eng_to_kor_train_40000_all.npy')
back_train_fin=[*back0,*back1,*back2,*back3,*back4]
back_train_fin=pd.DataFrame(back_train_fin,columns=['back_title'])

In [ ]:
back_train_fin=pd.concat([train,back_train_fin],axis=1)
back_train_fin

In [ ]:
# .?! 뒤 space가 없는 경우 공백추가(숫자사이는 제외) 후 '...' 처리 / 중복공백 제거
for i in range(0,len(back_train_fin)):
  back_train_fin.at[i,'back_title']=back_train_fin['back_title'][i].replace('U.S.','미국')
  back_train_fin.at[i,'back_title']=re.sub(r'([.?!/\\]+)(?![0-9])', r'\1 ', back_train_fin['back_title'][i]).replace('... ','...').replace('  ',' ').replace('.. ','..').strip()

In [ ]:
# If the proportion of Hangul in the total sentence length is 0.6 or higher, it is considered an abnormal translation and an attempt to re-translate it.
# -> 전체 문장길이에서 한글가 차지하는 비중이 0.6이상이면 이상번역으로 간주하고 재번역 시도

# Attempt to re-translate a translated sentence if the translated sentence has a ratio of less than 0.5 to the length of an existing sentence
# -> 번역된 문장이 기존 문장의 길이에 대한 비율이 0.5이하이면 재번역 시도
retrans_ind=[]

for i in tqdm(range(0,len(back_train_fin))):
  kor_ratio=len(re.sub('[a-zA-Z]','',back_train_fin['back_title'][i]))/(len(back_train_fin['back_title'][i])+1)
  if kor_ratio<0.6:
    retrans_ind.append(i)
  if len(back_train_fin['back_title'][i])/len(back_train_fin['title'][i])<=0.5:
    retrans_ind.append(i)

retrans_ind=list(set(retrans_ind))

In [ ]:
back_train_fin.iloc[retrans_ind,:]

In [ ]:
# Converting Chinese characters to Korean from re-translation targets
re_transed=[]
for i in back_train.iloc[retrans_ind,1]:
  temp=replace_all(i,d)
  re_transed.append(temp)

### Using pororo because it is not translated using Papago

In [ ]:
# Retranslating using pororo
mt = Pororo(task="translation", lang="multi")

retransed=[]

for k,i in tqdm(enumerate(re_transed)):
  eng=mt(i, src="ko", tgt="en")
  ko=mt(ko, src="en", tgt="ko")

  retransed.append(ko)
  if k%100==0:
    np.save(data_path+'re_trans.npy',retransed)
    print('{}th : '.format(k),retransed[k])

np.save(data_path+'re_trans.npy',retransed)

In [ ]:
retransed=np.load(data_path+'re_trans.npy')
back_train_fin.at[retrans_ind,'back_title']=retransed

In [ ]:
back_train_fin.iloc[retrans_ind,:]

In [ ]:
for i in range(0,len(back_train_fin)):
  back_train_fin.at[i,'back_title']=back_train_fin['back_title'][i].replace('... ','...').strip()

In [ ]:
back_train_fin.to_csv(data_path+'back_train_fin.csv',index=False)

# 4. Refinement of back translated data

+ Correct incorrectly translated parts
+ Calibrate sentence styles to resemble training data
+ Special Character Processing

+ 뉴스 타이틀은 특수 문자와 관련해서 일종의 규칙을 가지거나 문장의 어미가 완전하지 않은 경우가 많았습니다. 생성된 역번역 데이터에서, 이를 완전하게 따라가지는 못하는 특징들이 많이 보이다보니 문장길이가 원본데이터보다 지나치게 길어지는 등의 문제가 존재했습니다.

+ 예를 들면 원본 문장은 '...' 으로 끝나지 않는데 역번역 문장은 '.'으로 끝난다던지, '..했음' 이라는 어미를 가지는데 '했었다' 라는 어미를 가진다던지, 원본데이터에서는 사용하지 않는 특수문자를 사용하는 것 등, 이런 부분에 대해서 좀 주목했습니다. 따라서 해당 파트는 이러한 '문장 스타일'을 최대한 교정하는 부분입니다. 거의 모든 특수 문자와 문장 스타일의 교정을 위해 지속적으로 추가하다보니 코드가 난잡해졌습니다. 단순히 언급한 '문장  스타일' 을 원본 데이터의 분포에 맞게 교정하는 과정이라고 생각하시면 될듯합니다.

In [ ]:
back_train=pd.read_csv(data_path+'back_train_fin.csv')

In [ ]:
# Replace string
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

# During the translation, I found words that were not related to the original sentence were put in.
d = { "[프로젝트 리포트]": "", "[연구보고]": "","[시론]":"",'[특별기획]':"",'[행사참가기]':'','[학술발표회]':'','[논문초록]':'','[발언]':'','[기술정보]':'','[국제학술발표회]':'',
     '[서평]':'','[학회행사]':'','[행사보고]':'','[학술보고]':'','[세미나]':''}

In [ ]:
for i in range(0,len(back_train)):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('?',' ').replace('!',' ').replace(':',' ').replace(';','').replace('【·】','').replace('】','').replace('=','').replace(' → ','→').replace('→→','')

  back_train.at[i,'back_title']=replace_all(back_train.back_title[i],d)

  back_train.at[i,'back_title']=back_train['back_title'][i].replace('[','').replace(']',' ').replace('(',' ').replace(')',' ').replace('+','＋').replace('\'','').replace("\"","")

  back_train.at[i,'back_title']=re.sub(r'([\/]+)(?![0-9])', r'·', back_train['back_title'][i]).replace('· ','·')

  back_train.at[i,'back_title']=re.sub(r'([#]+[0-9])', r'\1위', back_train['back_title'][i]).replace('#','')
  if (back_train['title'][i].find('·')!=-1) & (back_train['back_title'][i].find('/')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('/','·')

  if (back_train['title'][i].find('·')!=-1) & (back_train['back_title'][i].find('-')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('-','·')
  if (back_train['title'][i].find('∼')!=-1) & (back_train['back_title'][i].find('-')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('-','∼')
  if (back_train['title'][i].find('∼')!=-1) & (back_train['back_title'][i].find('-')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('-','∼')
  if (back_train['title'][i].find('·')!=-1) & (back_train['back_title'][i].find(',')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace(',','·')

  back_train.at[i,'back_title']=back_train['back_title'][i].replace('북-미','북미').replace('미-중','미중').replace('원-달러','원달러').replace('미-러','미러').replace('K-리그','K리그').replace('convid-19','코로나19').replace('미-이란','미이란')\
  .replace('한-미','한미').replace('북-중','북중').replace('북-러','북러').replace('북-일','북일').replace('covid-19','코로나19').replace('COVID-19','코로나19').replace('Covid-19','코로나19').replace('페리 세월','세월호').replace('미스터 문','문 대통령')\
  .replace(' ↑','↑').replace('°C','↑').replace('iPhone','아이폰').replace('♬','').replace('ᄌ jeju jeju unique ','').replace('Galaxy Note ','갤노트').replace('Galaxy ','갤럭시').replace('Wi-Fi','와이파이').replace('습니다','다').replace('입니다','이다')\
  .replace('했습니다','했다').replace('나요','나').replace('어요','다').replace('아요','다').replace('까요','까').replace('서요','').replace('해요','')
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('    ',' ').replace('   ',' ').replace('  ',' ').replace('· ','·').strip()

In [ ]:
# Make sure have a sentence style that is as similar to the original one as possible
for i in tqdm(range(0,len(back_train))):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('...','@')
  back_train.at[i,'title']=back_train['title'][i].replace('...','@')

  if (back_train['title'][i].endswith('.')) & ~(back_train['back_title'][i].endswith('.')):
    back_train.at[i,'back_title']=back_train['back_title'][i]+'.'
  if ~(back_train['title'][i].endswith('.')) & (back_train['back_title'][i].endswith('.')):
    back_train.at[i,'back_title']=back_train['back_title'][i][:-1]

  back_train.at[i,'back_title']=back_train['back_title'][i].replace('@','...')
  back_train.at[i,'title']=back_train['title'][i].replace('@','...')

In [ ]:
# Define for special cases and correct them directly
for i in range(0,len(back_train)):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('. ','…').replace('D…C…','DC').replace('· D…A…L…T…',' ').replace('……','…').strip()

a=back_train[back_train['back_title'].str.contains('….….')].index
b=back_train[back_train['back_title'].str.contains('\.\.…')]['back_title'].index
c=back_train[(back_train['back_title'].str.contains('다\.'))&~(back_train['back_title'].str.contains('\.\.\.'))].index
d=back_train[back_train['back_title'].str.contains('.→.→.')]['back_title'].index
e=back_train[(back_train['back_title'].str.contains('뉴'))&(back_train['title'].str.contains('신간'))].index
f=back_train[(back_train['back_title'].str.contains('있어요'))&~(back_train['back_title'].str.endswith('있어요'))&~(back_train['back_title'].str.contains('있어요…'))&~(back_train['back_title'].str.contains('있어요 '))].index
g=back_train[~(back_train['title'].str.contains('합니다'))&(back_train['back_title'].str.contains('합니다'))].index
h=back_train[(back_train['title'].str.contains('종합'))&((back_train['back_title'].str.endswith('…합성'))|(back_train['back_title'].str.endswith('합성')))].index

for i in a:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('…','')
for i in b:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('..…','…')
for i in c:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('다.','다…')
for i in d:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('→','')
for i in e:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('뉴욕','신간').replace('뉴진',' 신간').replace('뉴','신간 ')  
for i in f:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('있어요','있다…')
for i in g:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('합니다','')
for i in h:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('…합성','종합').replace('합성','종합')

for i in range(0,len(back_train)):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('    ',' ').replace('   ',' ').replace('  ',' ').replace(' …','…').replace('… ','…').strip()

In [ ]:
back_train.sample(30)

In [ ]:
# Final back translated data used in training
back_train.to_csv(data_path+'back_train_fin2.csv',index=False)

# 5. Processing Confused sentence
- Extract singular sentences based on similarity for frequently confused labels

+ 제공된 데이터를 살펴보던 중, 특이한 점을 발견했는데 문장의 의미가 완전히 동일하거나 단어 하나의 조사 차이임에도 불구하고 라벨이 다른 이상한? 문장들을 다수 발견했습니다. 이부분은 실제 상황이라면 다시 정제를 하거나 라벨을 교정해주어야하는데 대회이다보니 test나 private test 데이터에 이러한 문장이 포함될 수 있다보니 단순한 제거는 불가능했습니다. 

+ '최정명' 님의 로그 위협탐지대회 코드에서 텍스트의 유사도를 기반으로 일정 유사도를 넘지만 라벨이 다른 경우, 학습시 강한 train signal을 주기위해 학습데이터에 포함하는 것을 보고 적용해보았습니다. 다만 전체 데이터에 대해 전부 적용한 것이 아니라, 모델 실험중에 confusion matrix를 보면 유난히 1,2,3 라벨에 대해 오분류율이 높은 것을 보고, 텍스트 유사도가 0.5이상인 해당 라벨을 가지는 문장들만을 대상으로 적용한 것이 전체에 대해 적용한 것보다 점수가 높았습니다. 

In [ ]:
# Calculate text simillarity
def return_similarity(a, b): 
    c = a.intersection(b)
    return float(len(c))/(len(a)+len(b)-len(c))

In [ ]:
tokenizer_bert=BertTokenizer.from_pretrained('klue/bert-base')

# Label = 1 or 2 or 3 in train data
train_data=train
train_1_2_3=train_data[(train_data['topic_idx']==1)|(train_data['topic_idx']==2)|(train_data['topic_idx']==3)]
train_1_2_3_index=train_data[(train_data['topic_idx']==1)|(train_data['topic_idx']==2)|(train_data['topic_idx']==3)].index

In [ ]:
# For calculate text similarity, tokenize all sentencies 
# Make new column 'token_list' to original train data
train_1_2_3['token_list']=''
train['token_list']=''
for i in train_1_2_3_index:
  token=tokenizer_bert.tokenize(train_1_2_3['title'][i])
  train_1_2_3['token_list'][i]=' '.join(token)
  train['token_list'][i]=' '.join(token)

In [ ]:
# Data frames for sentences with high text similarity
set_train_log = [set(log.split()) for log in train['token_list']]
sim_idx=[]
sim_i=[]
sim_j=[]
top_i=[]
top_j=[]

# If the similarity is higher than 0.5 with different labels, it is judged as a similar sentence
for i in tqdm(train_1_2_3_index):
    for j in train_1_2_3_index:
        if i==j:
            continue
        if ((return_similarity(set_train_log[i], set_train_log[j]))>=0.5) and (train_1_2_3['topic_idx'][i]!=train_1_2_3['topic_idx'][j]):
            top_i.append(train_1_2_3['topic_idx'][i])
            top_j.append(train_1_2_3['topic_idx'][j])
            sim_i.append(i)
            sim_j.append(j)

In [ ]:
# Labels are different despite semantically identical
crazy=pd.DataFrame({'sent1_idx':sim_i,'sent2_idx':sim_j,'sent1':train_1_2_3['title'][sim_i].values,'sent2':train_1_2_3['title'][sim_j].values,'top_1':top_i,'top_2':top_j})
crazy

In [ ]:
# Save odd data frame
np.save(data_path+'sim_1_2_3_0point5_klue_bert.npy',crazy)

# 6. Modeling & Prediction

+ 마지막으로 모델은 klue_roberta_large를 사용했습니다. Stratified 5-Fold를 이용해 각 80%의 train data로 학습시킨 모델 5개로test셋에 대해 앙상블한 것을 최종 파일로 제출했습니다. 학습데이터는 앞서 생성한 back translated data를 포함하였으며, 검증 데이터에는 해당 데이터를 사용하지않고 원데이터만 사용해 검증했습니다. 동시에 3번에서 언급한 confused sentence는 모든 fold의 학습 데이터에 포함하였습니다.

+ 모델링과정에서 신경쓴 것중 한가지가 optimizer부분입니다. 일반적으로 Adam을 사용해왔어서 초반에는 별생각없이 default값의 Adam을 사용했으나 과적합 양상이 심화되며 깊은 학습을 하지 못하는 것을 보고 transformer에서 제공하는 AdamWeightDecay를 사용해 learning rate를 낮추고 weight decay를 적용한 결과 더 높은 점수로 나아갈수 있었습니다.

In [ ]:
def reset_seeds(seed, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  

    np.random.seed(seed)
    random.seed(seed+100)
    tf.compat.v1.set_random_seed(seed+200)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  
    print("RANDOM SEEDS RESET {}".format(seed))  

SEED = 1514
reset_seeds(SEED)

In [ ]:
back_train=pd.read_csv(data_path+'back_train_fin2.csv')
back_train=back_train[['index','back_title','topic_idx']]
back_train.columns=train.columns

In [ ]:
# Loading indexes from strange data frames extracted earlier
crazy=pd.DataFrame(np.load(data_path+'sim_1_2_3_0point5_klue_bert.npy',allow_pickle=True),columns=['sent1_idx','sent2_idx','sent1','sent2','top_1','top_2'])
sim_idx=list(set([*crazy.sent1_idx.values,*crazy.sent1_idx.values]))

In [ ]:
# Cross validation, StratifiedKfold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]

# Unusual sentences must be included in the training data
# Seperate train set and validation set in each folds
for train_idx, valid_idx in skf.split(train, train['topic_idx']):
    train_idx = np.array(list(set(list(train_idx)+list(sim_idx))))
    valid_idx = np.array(list(set(set(valid_idx)-set(sim_idx))))
    folds.append((train_idx, valid_idx))

In [ ]:
def convert_data(data_df,case,mask_token):
    global tokenizer
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # tokenize
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, padding='max_length',truncation=True)
       
        # making input mask
        num_zeros = token.count(mask_token)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # making segment
        segment = [0]*SEQ_LEN
 
        # token, mask, segment
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        if case=='train':
          # label values
          targets.append(data_df[LABEL_COLUMN][i])
 
    # convert to array format    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    if case=='train':
      targets = np.array(targets)

    if case=='train':
       return [tokens, masks, segments], targets
    if case=='test':
       return [tokens, masks, segments]

 
# Load data and convert to bert input format
def load_data(pandas_dataframe,case,mask_token):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    if case=='train':
      data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
      data_x, data_y = convert_data(data_df,'train',mask_token)
      return data_x, data_y
    if case=='test':
      data_x = convert_data(data_df,'test',mask_token)
      return data_x

# Define max_len
SEQ_LEN = 30
DATA_COLUMN = "title"
LABEL_COLUMN = "topic_idx"
 
# train
train_x0, train_y0 = load_data(train.iloc[folds[0][0]].append(back_train.iloc[folds[0][0]]).reset_index(drop=True),'train',1)
train_x1, train_y1 = load_data(train.iloc[folds[1][0]].append(back_train.iloc[folds[1][0]]).reset_index(drop=True),'train',1)
train_x2, train_y2 = load_data(train.iloc[folds[2][0]].append(back_train.iloc[folds[2][0]]).reset_index(drop=True),'train',1)
train_x3, train_y3 = load_data(train.iloc[folds[3][0]].append(back_train.iloc[folds[3][0]]).reset_index(drop=True),'train',1)
train_x4, train_y4 = load_data(train.iloc[folds[4][0]].append(back_train.iloc[folds[4][0]]).reset_index(drop=True),'train',1)

valid_x0, valid_y0 = load_data(train.iloc[folds[0][1]].reset_index(drop=True),'train',1)
valid_x1, valid_y1 = load_data(train.iloc[folds[1][1]].reset_index(drop=True),'train',1)
valid_x2, valid_y2 = load_data(train.iloc[folds[2][1]].reset_index(drop=True),'train',1)
valid_x3, valid_y3 = load_data(train.iloc[folds[3][1]].reset_index(drop=True),'train',1)
valid_x4, valid_y4 = load_data(train.iloc[folds[4][1]].reset_index(drop=True),'train',1)

# test
test_x = load_data(test,'test',1)

In [ ]:
# Define RobertaModel using pretrained model
class Klue_RobertaClassifier(tf.keras.Model):
    def __init__(self, num_class):
        super(Klue_RobertaClassifier, self).__init__()

        self.bert = TFRobertaModel.from_pretrained("klue/roberta-large", from_pt=True)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range,seed=42), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        # outputs value : sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

klue_roberta_model = Klue_RobertaClassifier(num_class=7)

In [ ]:
# Learn about each fold around the for gate and store the best weights.
for i in range(5):
  print('########## Fold {} : \n'.format(i))

  klue_roberta_model = Klue_RobertaClassifier(num_class=7)

  # Defining loss function, optimizer and metric
  optimizer = AdamWeightDecay(1e-5,weight_decay_rate=1e-4)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  klue_roberta_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

  # Adding an ealrystop to prevent overfitting
  earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=2)

  checkpoint_path = os.path.join(model_path,'weight_klue_roberta_back_skf_fold_v0{}.h5'.format(i))
  cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

  # Training
  history = klue_roberta_model.fit(globals()['train_x{}'.format(i)],globals()['train_y{}'.format(i)], epochs=3, batch_size=32,
                             validation_data=(globals()['valid_x{}'.format(i)],globals()['valid_y{}'.format(i)]), callbacks=[earlystop_callback, cp_callback])

  klue_roberta_model.load_weights(model_path+'weight_klue_roberta_back_skf_fold_v0{}.h5'.format(i))

  preds=tf.argmax(klue_roberta_model.predict(globals()['valid_x{}'.format(i)]),axis=1)

  print('Validation set ACC: ',accuracy_score(globals()['valid_y{}'.format(i)],preds))
  print('Validation set Confusion Matrix: \n',confusion_matrix(globals()['valid_y{}'.format(i)],preds))

In [ ]:
# Load all weights and predict labels toward test data
for i in range(0,5):
 klue_roberta_model.load_weights(model_path+'weight_klue_roberta_back_skf_fold_v0{}.h5'.format(i))
 globals()['results_{}'.format(i)] = klue_roberta_model.predict(test_x)

# Save forecasts list toward test data
results_test_list=[results_0,results_1,results_2,results_3,results_4]
np.save(data_path+'results_klue_roberta_back_list_v01.npy',results_test_list)

In [ ]:
# Calculate forecasts for final test data by averaging all forecasts
test_pred=tf.argmax((results_test_list[0]+results_test_list[1]+results_test_list[2]+results_test_list[3]+results_test_list[4])/5,axis=1)
sample_submission['topic_idx']=test_pred
pd.merge(test,sample_submission).head(30)

In [ ]:
# Final submission
sample_submission.to_csv(os.path.join(sub_path,'klue_roberta_back_skf_v01.csv'), index=False)